## 0. 라이브러리 다운로드

### 라이브러리 설명

- 오디오 처리: numpy, librosa, soundfile, pydub
- 모델 실행: torch, transformers, accelerate
- 응용 프로그램 구성: langchain, sentence-transformers
- 데이터셋: datasets

## 설치한 라이브러리들 설명  

### 핵심 AI 및 NLP 관련 라이브러리
1. `torch`  
PyTorch는 딥러닝 모델을 구축하고 훈련하는 데 사용하는 오픈소스 라이브러리입니다.  
GPU 가속을 지원하며, 특히 자연어 처리와 컴퓨터 비전에서 널리 사용됩니다.  
Transformers 모델을 실행하거나 커스텀 모델을 훈련할 때 사용됩니다.  

2. `transformers`  
Hugging Face에서 제공하는 라이브러리로, 사전 학습된 NLP 모델(BERT, GPT, Whisper 등)을 쉽게 사용할 수 있습니다.  
Whisper 모델도 이 라이브러리를 통해 로드하여 사용합니다.  
Whisper 모델로 음성 데이터를 텍스트로 변환하는 데 사용됩니다.  

3. `accelerate`  
Hugging Face에서 제공하는 라이브러리로, 모델 훈련과 추론을 가속화하고 여러 디바이스(CPU, GPU, TPU 등)를 활용하도록 지원합니다.  
Whisper나 기타 Transformer 기반 모델을 실행 시 성능 최적화에 사용됩니다.  

4. `langchain`  
대규모 언어 모델(LLM)을 활용한 애플리케이션을 구축하는 프레임워크입니다.  
여러 NLP 작업(질의응답, 대화 생성 등)을 연결하는 워크플로우를 구성할 때 사용됩니다.  
LLM 기반 애플리케이션 개발에 사용됩니다. Whisper와 결합하여 음성 인식 후 처리 로직을 작성할 때 유용할 수 있습니다.  

1. `sentence-transformers`  
문장 수준의 임베딩(벡터 표현)을 생성하는 데 사용되는 라이브러리입니다.  
텍스트 데이터의 유사도 측정이나 검색 작업에 널리 사용됩니다.  
Whisper로 변환된 텍스트 데이터를 처리하거나 분석하는 데 사용될 수 있습니다.  


### 오디오 처리 및 데이터 관련 라이브러리  
1. `numpy==1.23.4`  
Python의 대표적인 수치 계산 라이브러리로, 행렬 연산 및 고성능 배열 처리를 지원합니다.  
Whisper 모델 및 오디오 데이터 처리에서 핵심적인 역할을 합니다.  
PCM 데이터를 처리하거나 librosa와 함께 오디오 데이터 배열을 조작할 때 사용됩니다.  
(`librosa`와 호환하기 위해 버전을 지정해주었습니다. 2.0.0 버전도 가능)  

2. `librosa`  
오디오 분석과 신호 처리를 위한 라이브러리입니다.  
오디오 데이터를 주파수 영역(Mel Spectrogram)으로 변환하는 등 Whisper 모델과 직접적으로 연관이 있습니다.  
Whisper 모델에 입력으로 제공되는 데이터를 전처리하거나 변환하는 데 사용됩니다.  

3. `soundfile`  
오디오 파일을 읽고 쓰는 데 사용되는 라이브러리입니다.  
`librosa`가 내부적으로 의존합니다.  
오디오 데이터를 로드하거나 저장할 때 사용됩니다.  

4. `pydub`  
오디오 데이터를 자르거나 합치는 등의 작업을 지원하는 라이브러리입니다.  
ffmpeg와 함께 동작하며, Whisper에 입력으로 제공할 오디오를 준비하는 데 유용합니다.  
긴 오디오 데이터를 30초 단위로 분할하거나 특정 포맷으로 변환할 때 사용됩니다.  

In [1]:
# !pip install torch transformers accelerate langchain sentence-transformers

  Using cached langchain-0.3.14-py3-none-any.whl.metadata (7.1 kB)
  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
  Using cached SQLAlchemy-2.0.36-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached langchain_core-0.3.29-py3-none-any.whl.metadata (6.3 kB)
  Using cached langchain_text_splitters-0.3.4-py3-none-any.whl.metadata (2.3 kB)
  Using cached langsmith-0.2.10-py3-none-any.whl.metadata (14 kB)
  Using cached pydantic-2.10.4-py3-none-any.whl.metadata (29 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached scikit_learn-1.6.0-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached scipy-1.15.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pillow-11.1.0-cp312-cp312-win_amd64.whl.metadata (9.3 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached orjson-3.10.13-cp312-cp312-win_amd64.whl.metadata (42 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadat

In [4]:
# !pip install numpy==1.23.4 librosa soundfile pydub

  Using cached numpy-1.23.4.tar.gz (10.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "C:\Users\dm705\AppData\Local\pypoetry\Cache\virtualenvs\ai_\xec쁺\xec뼱\xeb\x8c\x80\xed솕-UlUCYZce-py3.12\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "C:\Users\dm705\AppData\Local\pypoetry\Cache\virtualenvs\ai_\xec쁺\xec뼱\xeb\x8c\x80\xed솕-UlUCYZce-py3.12\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\dm705\AppData\Local\pypoetry\Cache\virtualenvs\ai_\xec쁺\xec뼱\xeb\x8c\x80\xed솕-UlUCYZce-py3.12\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 112, in get_requires_for_build_wheel
     

In [3]:
# !pip install librosa

  Using cached librosa-0.10.2.post1-py3-none-any.whl.metadata (8.6 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached numba-0.60.0-cp312-cp312-win_amd64.whl.metadata (2.8 kB)
  Using cached soundfile-0.13.0-py2.py3-none-win_amd64.whl.metadata (16 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached soxr-0.5.0.post1-cp312-abi3-win_amd64.whl.metadata (5.6 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
  Using cached msgpack-1.1.0-cp312-cp312-win_amd64.whl.metadata (8.6 kB)
  Using cached llvmlite-0.43.0-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
Using cached librosa-0.10.2.post1-py3-none-any.whl (260 kB)
Using cached audioread-3.0.1-py3-none-any.whl (23 kB)
Using cached lazy_loader-0.4-py3-none-any.whl (12 kB)
Using cached msgpack-1.1.0-cp312-cp312-win_amd64.whl (75 kB)
Using cached numba-0.60.0-cp312-cp312-win_amd64.whl (2.7 MB)
Using cached pooch-1.8.2-py3-none-any.whl (64 kB)
Using cached soundfile-0.

## 1. 모듈 불러오기

In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
import librosa

### 1.1. 모델 불러오기 및 파이프라인 설정

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype,
    # low_cpu_mem_usage=True, 
    use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

Device set to use cpu


## 2. 데이터셋 활용
- datasets / distil-whisper/librispeech_long
  - 허깅페이스에서 제공하는 음성 데이터셋으로, 주로 ASR평가를 위해 사용
  - 데이터 샘플
  
  ```python
  {
    "audio": {
        "path": "파일 경로 또는 파일 ID",
        "array": array([...]),  # PCM 데이터 배열
        "sampling_rate": 16000  # 샘플링 속도
    }
  }
  ```

In [3]:
dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")

In [4]:
dataset[0]

{'audio': {'path': '0d38672e0bbdbdc460af55b8bb84a15b2730db2819f2af64f9c777d4d586f2de',
  'array': array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00024414, 0.00048828,
         0.0005188 ]),
  'sampling_rate': 16000}}

In [8]:
sample = dataset[0]['audio']

### 2.1. 데이터셋 변환 (PCM -> pydub.AudioSegment)
- PCM(`Pulse Code Modulation`): 디지털 오디오 데이터의 기본 형식  

- **특징**
  - 1. 비압축 데이터
    - 압축 코덱(MP3, AAC)과 달리 압축되지 않은 원시의 데이터
  - 2. 샘플링
    - 아날로그 오디오 신호를 일정한 간격으로 측정하여 디지털 값으로 변환
  - 3. 샘플 폭
    - PCM이 차지하는 비트 수를 나타내는데, 16비트, 8비트 같은 게 있음.
  - 4. 채널
    - 오디오가 모노(1채널)인지, 스테레오(2채널)인지 결정.

- **장단점**
  - 장점
    - 1. 고품질 데이터
      - 압축되지 않은 원본 데이터를 제공하므로, 품질 손실이 없음
    - 2. 표준화
      - 오디오 처리 라이브러리나 딥러닝 모델에서 기본적으로 사용 가능
  - 단점
    - 1. 큰 파일 크기
      - 압축되지 않았기에 크기가 큼
    - 2. 직접 사용에 불편
      - 사람이 바로 이해하거나 사용 불가능한 형태로, 추가 처리가 필요함.
    

샘플 데이터를 짧게 보면

```python
{
    'audio':{
        'path': "파일 경로 또는 파일 ID",
        'array' : [0.0023, 0.0035, -0.0046, ...], #PCM 데이터 배열
        'sampling_rate' : 16000, # 샘플링 속도 (16,000Hz)
        'channels' : 1, # 모노(1채널)
    }
}
```

In [11]:
sample

{'path': '0d38672e0bbdbdc460af55b8bb84a15b2730db2819f2af64f9c777d4d586f2de',
 'array': array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00024414, 0.00048828,
        0.0005188 ]),
 'sampling_rate': 16000}

In [13]:
import numpy as np
from pydub import AudioSegment

# 오디오 파일 로드

# 오디오 데이터 추출 및 AudioSegment 변환
audio_array = sample['array']
sampling_rate = sample['sampling_rate']

# PCM 데이터를 16비트 정수로 변환
audio_data = np.array(audio_array * 32767, dtype=np.int16)

# AudioSegment로 변환
audio = AudioSegment(
    audio_data.tobytes(),
    frame_rate=sampling_rate,
    sample_width=audio_data.dtype.itemsize,
    channels=1
)

# 오디오 분할 (30초 단위)
chunk_length_ms = 30 * 1000  # 30초
chunks = [audio[i:i + chunk_length_ms] for i in range(0, len(audio), chunk_length_ms)]

# 각 분할 저장
for i, chunk in enumerate(chunks):
    chunk.export(f"chunk_{i}.wav", format="wav")
    print(f"chunk_{i}.wav 저장 완료")

chunk_0.wav 저장 완료
chunk_1.wav 저장 완료
chunk_2.wav 저장 완료


In [15]:
chunks

In [19]:
from pydub.playback import play
play(chunk[0])  # 이런 식으로 재생 가능

기존 임시 파일 경로: C:\Users\dm705\AppData\Local\Temp
